In [ ]:
import bs4
import urllib.request as req
from urllib.parse import quote
import requests 
import pandas as pd

def getdata(board=None,kw=None,pages=2000):
  count=0
  push_lst = []
  board_lst = []
  author_lst = []
  board_lst = []
  title_lst = []
  date_lst = []
  url_lst = []
  content_lst = []
  url = f"https://www.ptt.cc/bbs/{board}/index.html"

  n=0

  while True:
    print("------ <第"+str(n+1)+"頁資料>------")
 
    request=req.Request(url,headers={
        "cookie":"over18=1",
        "user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36"
    })
    
    try:
      with req.urlopen(request) as response:
        data=response.read().decode("utf-8")
      root=bs4.BeautifulSoup(data,"html.parser")
      titles=root.select("div.title")
    except:
      print(False)
      count+=1
      if count == 200:
        print(url)
        break
      else:
        continue
    
    #Obtain title and data.
    for title in titles:
        try:
          url = "https://www.ptt.cc"+title.a["href"]

          my_headers = {"cookie":"over18=1",
          "user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36"
          }
          response2 = requests.get(url, headers = my_headers)
          soup = bs4.BeautifulSoup(response2.text,"html.parser")

          header = soup.find_all('span','article-meta-value')

          push = soup.find_all('div' , 'push')
          date = header[3].text
          tmp = [i.text for i in push]
          push_lst.append(tmp)
          title_lst.append(title.a.get_text())
          date_lst.append(date)
          url_lst.append(url)
        except:
          print("Error happened")
          continue
    
    #Next page
    try:
      nextlink=root.find("a",string="‹ 上頁")
      url=str("https://www.ptt.cc"+nextlink["href"]) 
    except:
      print(url)
      break
        
    if n == pages-1:
      break
    n+=1
  
  df = pd.DataFrame(list(zip(title_lst, date_lst, url_lst, push_lst)),columns=["標題","日期","網址","推文內容"])
  df.to_csv(f'C:\\Users\\User\\Desktop\\{kw}.csv', index=False,encoding='utf-8')
  return


#Get datas
getdata("swim","swim",1130)
getdata("GO","GO",500)
getdata("Hiking","Hiking",1000)
getdata("Badminton","Badminton",1600)
getdata("Tabletennis","tabletennis",1800)